# Assignment #1: Building an inverted index

## Objectives

The objectives of this assignment are to:
* Write a program that collects all the words from a set of documents
* Build an index from the words
* Represent a document using the Tf.Idf value
* Write a short report of 1 to 2 pages on the assignment
* Read a short text on an industrial system

## Description

### Outline

Conceptually, an index consists of rows with one word per row and and the list of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.
<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>

### Preparation

<p>You will create an index for a corpus of Selma Lagerlöf's works:</p>
<ol>
            <li>Download the <a href="https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip">Selma
                folder</a> and uncompress it. It contains novels by <a href="https://sv.wikipedia.org/wiki/Selma_Lagerl%C3%B6f">
                Selma Lagerlöf</a>. The text of these novels was extracted
                from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
                <a href="https://litteraturbanken.se/">Litteraturbanken</a>.
            </li>
            <li>Read the description of the tf-idf measure on Wikipedia
                (<a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf">https://en.wikipedia.org/wiki/Tf-idf</a>).
            </li>
</ol>

### Running the Indexer

Your final program will take a corpus as input (here the Selma novels) and produce an index of all the words with their positions. you should be able to run it this way:
<pre>$ python indexer.py folder_name</pre>
In this lab, you will write it in a Jupyter Notebook. The conversion into a Python program is left as an optional exercise.

## Programming the Indexer

To make programming easier, you will split this exercise into five steps:
1. Index one file;
2. Read the content of a folder
3. Create a master index for all the files
4. Use tfidf to represent the documents (novels)
5. Compare the documents of a collection

You will use dictionaries to represent the postings.

### Indexing one file

#### Description

<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
            <li>The index file will contain all the unique words in the document,
                where each word is associated with the list of its positions in the document.
            </li>
            <li>You will represent this index as a dictionary, where the keys will be the words, and
                the values, the lists of positions
            </li>
            <li>As words, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, you will use Unicode regular expressions. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The word positions will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use <tt>finditer()</tt> to find the positions of the words.
                This will return you match objects,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
            <li>You will use the pickle package to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>

<p>Below is an excerpt of the index of the bannlyst text for the words <i>gjord</i>, <i>uppklarnande</i>, and
            <i>stjärnor</i>. The data is stored in a dictionary:
        </p>
        <pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
        </pre>
        <p>The word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at
            position 8617, and <i>stjärnor</i>, once at position 8641.
        </p>

#### Imports

Some imports you could need. Add others as needed

In [1]:
import regex as re
import os
import regex as re
import pickle
import sys
import math
import time

#### Writing a tokenizer 

Write a Unicode regular expression to find words defined as sequences of letters.

In [2]:
# Write your regex here
regex = '\p{L}+'

In [3]:
re.findall(regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

Using `regex`, write a function to tokenize a text. Return their positions.

In [4]:
# Write your code here
def tokenize(string):
    return re.findall(regex, string)

In [5]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

#### Extracting indices

Write a function to extract the indices from the list of tokens (words). Return a dictionary, where the keys will be the tokens (words), and the keys a list of positions.

In [6]:
# Write your code here
def text_to_idx(tokens):
    idx = {}
    index = 0
    
    for word in tokens:
        if word in idx:
            idx[word] = idx[word] + [index]
        else:
            idx[word] = [index]
        index = index + len(word);
        
    return idx
            


In [7]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 22],
 'gång': [2],
 'hade': [6],
 'de': [10],
 'på': [12],
 'mårbacka': [14],
 'barnpiga': [24],
 'som': [32],
 'hette': [35],
 'back': [40],
 'kajsa': [44]}

#### Reading one file

Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`

In [8]:
# Write your code here
file = open('Selma/marbacka.txt')
text = file.read().lower().strip()
tokens = tokenize(text)
idx = text_to_idx(tokens)




In [9]:
idx['mårbacka']

[13,
 100,
 587,
 1331,
 2032,
 2613,
 11850,
 12079,
 21874,
 33120,
 38608,
 39649,
 40943,
 47262,
 49789,
 50982,
 52758,
 52875,
 53252,
 53272,
 54387,
 56035,
 56645,
 58281,
 58367,
 58659,
 60202,
 60413,
 60616,
 60914,
 61139,
 61259,
 61763,
 63382,
 64332,
 64509,
 66411,
 66745,
 67115,
 70710,
 76318,
 77467,
 78876,
 82513,
 82694,
 83033,
 85421,
 85755,
 88667,
 89356,
 90365,
 90821,
 91015,
 91439,
 91587,
 95705,
 97982,
 99230,
 99332,
 101061,
 102796,
 103535,
 104246,
 107509,
 107724,
 108276,
 109301,
 110443,
 111718,
 115168,
 118809,
 121081,
 121726,
 121962,
 122319,
 122717,
 124042,
 125368,
 125588,
 126413,
 127157,
 130148,
 132021,
 132181,
 132625,
 133591,
 135474,
 142662,
 142970,
 146136,
 148633,
 149794,
 149884,
 151523,
 154256,
 158914,
 161923,
 163077,
 163545,
 164730,
 165242,
 170744,
 172699,
 173882,
 173995,
 174277,
 174540,
 176130,
 180332,
 180408,
 181336,
 181864,
 182408,
 184186,
 185582,
 186179,
 188743,
 190049,
 190411

#### Saving the index

Save your index in a file so that you can reuse it. Use the pickel module.

In [13]:
# Write your code here
pickle.dump(idx, open('marbacka.p','wb'))


In [15]:
# Write your code here
idx = pickle.load(open('marbacka.p','rb'))

In [16]:
idx['mårbacka']

[13,
 100,
 587,
 1331,
 2032,
 2613,
 11850,
 12079,
 21874,
 33120,
 38608,
 39649,
 40943,
 47262,
 49789,
 50982,
 52758,
 52875,
 53252,
 53272,
 54387,
 56035,
 56645,
 58281,
 58367,
 58659,
 60202,
 60413,
 60616,
 60914,
 61139,
 61259,
 61763,
 63382,
 64332,
 64509,
 66411,
 66745,
 67115,
 70710,
 76318,
 77467,
 78876,
 82513,
 82694,
 83033,
 85421,
 85755,
 88667,
 89356,
 90365,
 90821,
 91015,
 91439,
 91587,
 95705,
 97982,
 99230,
 99332,
 101061,
 102796,
 103535,
 104246,
 107509,
 107724,
 108276,
 109301,
 110443,
 111718,
 115168,
 118809,
 121081,
 121726,
 121962,
 122319,
 122717,
 124042,
 125368,
 125588,
 126413,
 127157,
 130148,
 132021,
 132181,
 132625,
 133591,
 135474,
 142662,
 142970,
 146136,
 148633,
 149794,
 149884,
 151523,
 154256,
 158914,
 161923,
 163077,
 163545,
 164730,
 165242,
 170744,
 172699,
 173882,
 173995,
 174277,
 174540,
 176130,
 180332,
 180408,
 181336,
 181864,
 182408,
 184186,
 185582,
 186179,
 188743,
 190049,
 190411

### Reading the content of a folder

<p>Write a function that reads all the files in a folder with a specific suffix (txt). You will need the Python
            os package,
            see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You
            will return the file names in a list.
        </p>

You can reuse this function:

In [28]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [27]:
# Write your code here



### Creating a master index

<p>Complete your program with the creation of master index, where you will associate each word
            of the corpus with the files, where it occur and its positions. (a posting list)
        </p>
        <p>Below is an except of the master index with the words <i>samlar</i> and <i>ände</i>:
        </p>

In [19]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

{'samlar': {'troll.txt': [641880, 654233],
  'nils.txt': [51805, 118943],
  'osynliga.txt': [399121],
  'gosta.txt': [313784, 409998, 538165]},
 'ände': {'troll.txt': [39562, 650112],
  'kejsaren.txt': [50171],
  'marbacka.txt': [370324],
  'nils.txt': [1794],
  'osynliga.txt': [272144]}}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
            313784, 409998, and 538165.

In [ ]:
# write your code here


In [ ]:
master_index['samlar']

In [ ]:
master_index['mårbacka']

Save your master index in a file and read it again

In [ ]:
# Write your code here


In [ ]:
master_index['samlar']

#### Concordances

Write a function to extract the concordances of a word within a window of `window` characters

In [ ]:
# Write your code here


In [ ]:
concordance('samlar', master_index, corpus_files, 25)

### Representing Documents with tf-idf

<p>Once you have created the index, you will represent each document in your corpus as a word vector.
            You will define the value of a word in a document with the tf-idf
            metric. Tf will be the relative frequency of the term in the document and idf, the logarithm base 10 of the
            inverse document
            frequency.
        </p>
        <p>You have below the tf-idf values for a few words. In our example, the word <i>gås</i> has the value
            0 in bannlyst.txt and the value 0.000101001964 in nils.txt
        </p>
        <pre>
troll.txt
	känna	 0.0
	gås	 0.0
	nils	 2.148161748868631e-06
	et	 0.0
kejsaren.txt
	känna	 0.0
	gås	 0.0
	nils	 8.08284798629935e-06
	et	 8.273225429362848e-05
marbacka.txt
	känna	 0.0
	gås	 0.0
	nils	 7.582276564686669e-06
	et	 9.70107989686256e-06
herrgard.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
nils.txt
	känna	 0.0
	gås	 0.00010100196417506702
	nils	 0.00010164426900380124
	et	 0.0
osynliga.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
jerusalem.txt
	känna	 0.0
	gås	 0.0
	nils	 4.968292117670952e-06
	et	 0.0
bannlyst.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
gosta.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
        </pre>


In [ ]:
# Write your code here


In [ ]:
tfidf['troll.txt']['känna']

In [ ]:
tfidf['troll.txt']['nils']

### Comparing Documents

Using the cosine similarity, compare all the pairs of documents with their tfidf representation and present
            your results in a matrix. You will include this matrix in your report.

#### Cosine similarity

Write a function computing the cosine similarity between two documents

In [ ]:
# Write your code here


#### Similarity matrix

Compute the similarity matrix between the documents of the corpus

In [ ]:
# Write your code here


Give the name of the two novels that are the most similar.

In [ ]:
print("Most similar:", most_sim_f1, most_sim_f2, "Similarity:", max_similarity)

<h2>Reading</h2>

<p>Read the text:
            <i>Challenges in Building Large-Scale Information Retrieval Systems</i>
            about the history of
            <a href="https://research.google.com/people/jeff/WSDM09-keynote.pdf">Google indexing</a>
            by
            <a href="https://research.google.com/pubs/jeff.html">Jeff Dean</a>.
            In your report, tell how your index encoding is related to what Google did.
            You must identify the slide where you have the most similar indexing technique and write the
            slide number in your report.
        </p>